# Suma de Riemann

In [ ]:
import math
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from bqplot import Figure, Scatter, Axis, LinearScale

# -------------------------
# TUS FUNCIONES ORIGINALES
# -------------------------
entradas = {
    "sen": "math.sin",
    "sin": "math.sin",
    "cos": "math.cos",
    "tan": "math.tan",
    "e^": "math.exp",
    "ln": "math.log",
    "log": "math.log10",
    "x^": "x**"
}

def sanitizar(s):
    for clave, valor in entradas.items():
        s = s.replace(clave, valor)
    return s

def sumaInferior(funcion, desde, hasta, rectangulos):
    if rectangulos <= 0:
        raise ValueError("En la primaria no te enseñaron que no podes dividir algo por cero?")
    if hasta <= desde:
        raise ValueError("Estas en tu casa y para ir a la Uni, vas desde la Uni a tu Casa?")

    a, b, n = desde, hasta, rectangulos
    delta = (b - a) / n
    suma = 0
    for k in range(n):
        izq = a + k * delta
        der = izq + delta
        m_k = min(funcion(izq), funcion(der))
        suma += m_k * delta
    return suma

def sumaSuperior(funcion, desde, hasta, rectangulos):
    if rectangulos <= 0:
        raise ValueError("En la primaria no te enseñaron que no podes dividir algo por cero?")
    if hasta <= desde:
        raise ValueError("Estas en tu casa y para ir a la Uni, vas desde la Uni a tu Casa?")

    a, b, n = desde, hasta, rectangulos
    delta = (b - a) / n
    suma = 0
    for k in range(n):
        izq = a + k * delta
        der = izq + delta
        M_k = max(funcion(izq), funcion(der))
        suma += M_k * delta
    return suma

def SumaSupSumaInf(funcion, minimo, maximo, eps):
    N = 1
    data = []
    while True:
        inf = sumaInferior(funcion, minimo, maximo, N)
        sup = sumaSuperior(funcion, minimo, maximo, N)
        integral = (inf + sup) / 2
        dif = abs(sup - inf)

        data.append({
            "Rectángulos": N,
            "Suma Inferior": inf,
            "Integral": integral,
            "Suma Superior": sup,
            "Diferencia": dif
        })

        if dif < eps:
            break
        N += 1
    return data


# -------------------------
# WIDGETS INTERACTIVOS
# -------------------------
input_funcion = widgets.Text(
    value="x**2",
    description="f(x):",
    placeholder="Ej: sen(x)+3"
)

input_a = widgets.FloatText(value=0, description="Desde (a):")
input_b = widgets.FloatText(value=2, description="Hasta (b):")
input_eps = widgets.FloatText(value=0.1, description="ε:")

boton = widgets.Button(description="Calcular", button_style="success")
salida = widgets.Output()

# -------------------------
# CONFIGURACIÓN DEL GRÁFICO bqplot
# -------------------------
x_scale = LinearScale()
y_scale = LinearScale()

scatter_inf = Scatter(x=[], y=[], scales={"x": x_scale, "y": y_scale}, colors=["blue"])
scatter_int = Scatter(x=[], y=[], scales={"x": x_scale, "y": y_scale}, colors=["red"])
scatter_sup = Scatter(x=[], y=[], scales={"x": x_scale, "y": y_scale}, colors=["green"])

fig = Figure(
    marks=[scatter_inf, scatter_int, scatter_sup],
    axes=[
        Axis(scale=x_scale, label="N"),
        Axis(scale=y_scale, orientation="vertical", label="Valores", side="left")
    ],
    title="Convergencia de Sumas de Riemann",
    layout={"width": "600px", "height": "400px"}
)


# -------------------------
# ACCIÓN DEL BOTÓN
# -------------------------
def ejecutar(_):
    with salida:
        salida.clear_output()

        try:
            # Función ingresada por el usuario
            expr = sanitizar(input_funcion.value)
            funcion = lambda x: eval(expr, {"math": math, "x": x})

            a = float(input_a.value)
            b = float(input_b.value)
            eps = float(input_eps.value)

            datos = pd.DataFrame(SumaSupSumaInf(funcion, a, b, eps))

            display(datos)

            # Actualizar gráfico
            scatter_inf.x = datos["Rectángulos"]
            scatter_inf.y = datos["Suma Inferior"]

            scatter_int.x = datos["Rectángulos"]
            scatter_int.y = datos["Integral"]

            scatter_sup.x = datos["Rectángulos"]
            scatter_sup.y = datos["Suma Superior"]

            display(fig)

        except Exception as e:
            print("Error:", e)

boton.on_click(ejecutar)

# MOSTRAR UI
widgets.VBox([
    widgets.HBox([input_funcion]),
    widgets.HBox([input_a, input_b, input_eps]),
    boton,
    salida
])
